# Introducing molecular dynamics

This hands-on workshop introduces the basic concepts of HOOMD-blue and will teach you how to run a molecular dynamics simulation. For more details, information that may be helpful when completing the exercises, see the [HOOMD-blue documentation](https://hoomd-blue.readthedocs.io/).

**Google Colab users:** Click the "Copy to Drive" button in the toolbar to save this notebook and any edits that you make to your Google Drive.

## Utility code

Please ignore the content of these cells, they provide some helpful methods but are not part of the workshop.

Run them one at a time, then click the arrow by "Utility code" to hide them.

In [ ]:
# This cell installs HOOMD-blue in Google Colab.
try:
    import google

    !pip install -q condacolab
    import condacolab

    condacolab.install_from_url(
        'https://github.com/glotzerlab/hoomd-workshop/releases/download/2023.0.0/hoomd-workshop-2023.0.0-Linux-x86_64.sh'
    )
except ModuleNotFoundError:
    pass

In [ ]:
import math
import warnings

import fresnel
import freud
import gsd.hoomd
import hoomd
import IPython
import matplotlib
import numpy
import packaging.version

device = fresnel.Device()
tracer = fresnel.tracer.Path(device=device, w=400, h=400)

FRESNEL_MIN_VERSION = packaging.version.parse("0.13.0")
FRESNEL_MAX_VERSION = packaging.version.parse("0.14.0")


def render(snapshot):
    if ('version' not in dir(fresnel) or packaging.version.parse(
            fresnel.version.version) < FRESNEL_MIN_VERSION
            or packaging.version.parse(
                fresnel.version.version) >= FRESNEL_MAX_VERSION):
        warnings.warn(
            f"Unsupported fresnel version {fresnel.version.version} - expect errors."
        )
    L = snapshot.configuration.box[0]
    scene = fresnel.Scene(device)
    geometry = fresnel.geometry.Sphere(scene,
                                       N=len(snapshot.particles.position),
                                       radius=0.5)
    geometry.material = fresnel.material.Material(color=fresnel.color.linear(
        [252 / 255, 209 / 255, 1 / 255]),
                                                  roughness=0.5)
    geometry.position[:] = snapshot.particles.position[:]
    geometry.outline_width = 0.04

    if (hasattr(snapshot, 'log')
            and 'particles/md/pair/LJ/energies' in snapshot.log):
        energies = snapshot.log['particles/md/pair/LJ/energies']
        mapper = matplotlib.cm.ScalarMappable(
            norm=matplotlib.colors.Normalize(vmin=numpy.min(energies),
                                             vmax=numpy.max(energies),
                                             clip=True),
            cmap=matplotlib.cm.get_cmap(name='viridis'))
        geometry.color[:] = fresnel.color.linear(mapper.to_rgba(energies))
        geometry.material.primitive_color_mix = 1

    box = fresnel.geometry.Box(scene, [L, L, L, 0, 0, 0], box_radius=.02)

    scene.lights = [
        fresnel.light.Light(direction=(0, 0, 1),
                            color=(0.8, 0.8, 0.8),
                            theta=math.pi),
        fresnel.light.Light(direction=(1, 1, 1),
                            color=(1.1, 1.1, 1.1),
                            theta=math.pi / 3)
    ]
    scene.camera = fresnel.camera.Orthographic(position=(L * 2, L, L * 2),
                                               look_at=(0, 0, 0),
                                               up=(0, 1, 0),
                                               height=L * 1.4 + 1)
    scene.background_color = (1, 1, 1)
    return IPython.display.Image(tracer.sample(scene, samples=20)._repr_png_())


warnings.filterwarnings('ignore', module='hoomd')

## Create the simulation and initial state

Just like we did with HPMC.

In [ ]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=2)

In [ ]:
snapshot = hoomd.Snapshot()
box, positions = freud.data.UnitCell.bcc().generate_system(num_replicas=4,
                                                           scale=1.5)
snapshot.particles.N = len(positions)
snapshot.particles.position[:] = positions
snapshot.particles.types = ['A']
snapshot.configuration.box = [box.Lx, box.Ly, box.Lz, 0, 0, 0]
render(snapshot)

In [ ]:
sim.create_state_from_snapshot(snapshot)

## Molecular dynamics classes

The MD integrator numerically integrates the equations of motion:

In [ ]:
integrator = hoomd.md.Integrator(dt=0.005)

You need a neighbor list to compte pairwise interactions:

In [ ]:
cell = hoomd.md.nlist.Cell(buffer=0.4)

Compute Lennard-Jones interaction forces:
$$V_{\mathrm{LJ}}(r) = 4 \varepsilon \left[ \left(\frac{\sigma}{r}\right)^{12} - \left(\frac{\sigma}{r}\right)^6 \right]$$

In [ ]:
lj = hoomd.md.pair.LJ(nlist=cell)
lj.params[('A', 'A')] = dict(epsilon=1, sigma=1)

Set `r_cut` to 2.5:

In [ ]:
# Add code.
lj.r_cut[('A', 'A')] = 2.5  # Solution

Add the `lj` force to the integrator's `forces` list:

In [ ]:
# Add code.
integrator.forces.append(lj)  # Solution

The integration **method** sets the specific equations of motion that apply to a subset of particles:

In [ ]:
all_particles = hoomd.filter.All()
kT = 1.5
nvt = hoomd.md.methods.NVT(kT=kT, filter=all_particles, tau=1.0)

Add `nvt` to the integrator's `methods` list:

In [ ]:
# Add code.
integrator.methods.append(nvt)  # Solution

Remember to add the integrator to the simulation operations!

In [ ]:
sim.operations += integrator

## Running a molecular dynamics simulation

HOOMD-blue defaults particle velocities to 0:

In [ ]:
print(sim.state.get_snapshot().particles.velocity[0:5, :])

Thermalize the velocities and integration method's degrees of freedom before running.

In [ ]:
sim.state.thermalize_particle_momenta(filter=all_particles, kT=kT)
sim.run(0)
nvt.thermalize_thermostat_dof()
print(sim.state.get_snapshot().particles.velocity[0:5, :])

Run the simulation:

In [ ]:
sim.run(100)

## Querying thermodynamic quantities

The `ThermodynamicQuantities` class computes properties of the simulation:

In [ ]:
thermo = hoomd.md.compute.ThermodynamicQuantities(filter=all_particles)
sim.operations += thermo

Use `thermo` to get the `potential_energy` of the system:

In [ ]:
# Add code.
thermo.potential_energy  # Solution.

## Logging scalar quantities

Use a Logger to monitor how properties of the simulation vary during the run:

In [ ]:
logger = hoomd.logging.Logger(categories=['scalar'])

Choose the specific quantities you want to log:

In [ ]:
logger += sim
logger.add(thermo, quantities=['potential_energy'])

Use the `Table` writer to display the logged values:

In [ ]:
table = hoomd.write.Table(trigger=hoomd.trigger.Periodic(100),
                          logger=logger,
                          max_header_len=5,
                          max_precision=2)
sim.operations += table

In [ ]:
sim.run(1000)

Remove the table writer:

In [ ]:
sim.operations -= table

## Logging array quantities

HOOMD-blue can log array quantities, such as the per particle energies `lj.energies`:

In [ ]:
lj.energies[0:5]

However, `Table` does not write array quantities. Create a new logger that accepts `'scalar'` and `'particle'` categories:

In [ ]:
# Add code.
array_logger = hoomd.logging.Logger(categories=['scalar',
                                                'particle'])  # Solution.

Then add the per-particle quantity `energies` from `lj` to it:

In [ ]:
# Add code.
array_logger.add(lj, quantities=['energies'])  # Solution.

Include the logged quantities in a GSD trajectory:

In [ ]:
gsd_writer = hoomd.write.GSD(trigger=hoomd.trigger.Periodic(100),
                             filename='trajectory.gsd',
                             log=array_logger,
                             mode='wb')
sim.operations += gsd_writer

Run the simulation to generate the file:

In [ ]:
sim.run(1000)

The values are available as numpy arrays:

In [ ]:
with gsd.hoomd.open('trajectory.gsd', 'rb') as trajectory:
    for frame in trajectory:
        energies = frame.log['particles/md/pair/LJ/energies']
        print(energies.mean(), energies.std())

Map the potential energy to particle color:

In [ ]:
with gsd.hoomd.open('trajectory.gsd', 'rb') as trajectory:
    image = render(trajectory[-1])
image